# Processing brat file

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read txt file
filename = 626296
file_txt = '../data/hpo_ann/letters/' + str(filename) + '.txt'
with open(file_txt, 'r') as f:
    lines = f.readlines()

lines[15:20]

['With best wishes,\n', '\n', 'Yours sincerely,\n', '\n', '\n']

In [3]:
file_ann = '../data/hpo_ann/letters/' + str(filename) + '.ann'
anns = pd.read_csv(file_ann, sep='\t', names=['id', '_type', 'entity'])
anns

,id,_type,entity
0,T4,EyeDiseases 912 918,Myopia
1,T31,EyeDiseases 267 273,Myopia
2,T34,EyeDiseases 344 354,low vision
3,T61,EyeDiseases 1543 1557,lens opacities
4,T63,EyeDiseases 1582 1588,Myopia


In [4]:
anns['start'] = anns['_type'].apply(lambda x: x.split(' ')[-2])
anns['end'] = anns['_type'].apply(lambda x: x.split(' ')[-1])
anns['type'] = anns['_type'].apply(lambda x: x.split(' ')[0])
anns = anns.drop(columns=['_type'])

In [5]:
anns

,id,entity,start,end,type
0,T4,Myopia,912,918,EyeDiseases
1,T31,Myopia,267,273,EyeDiseases
2,T34,low vision,344,354,EyeDiseases
3,T61,lens opacities,1543,1557,EyeDiseases
4,T63,Myopia,1582,1588,EyeDiseases


In [6]:
# remove all anns whose type is not 'EyeDisease'
anns = anns.loc[anns['type'] == 'EyeDiseases']

In [7]:
# convert start to int for sorting
anns['start'] = anns['start'].astype('int') 
anns['end'] = anns['end'].astype('int')

In [8]:
anns_sorted = anns.sort_values(by=['start'], ignore_index=True)
anns_sorted

,id,entity,start,end,type
0,T31,Myopia,267,273,EyeDiseases
1,T34,low vision,344,354,EyeDiseases
2,T4,Myopia,912,918,EyeDiseases
3,T61,lens opacities,1543,1557,EyeDiseases
4,T63,Myopia,1582,1588,EyeDiseases


In [9]:
# Test with an example
lines_test = ['vision\n', '\n', 'loss\n']

start_sent = 0
end_sent = 0
for line in lines_test:
    end_sent = start_sent + len(line) - 1
    print(f'{line} \t {start_sent} \t {end_sent}')
    start_sent =  end_sent + 1
    

vision
 	 0 	 6

 	 7 	 7
loss
 	 8 	 12


In [10]:
# Main processing
N = len(anns)
start_sent = 0
end_sent = 0
i_start = 0
sents = []
start_ann = anns_sorted['start'][i_start]

for line in lines:
    end_sent = start_sent + len(line) - 1
    
    while start_ann >= start_sent and start_ann <= end_sent:
        print(f"Target sent detected: \n {line} | {i_start} | {start_ann} | {anns_sorted['entity'][i_start]}\n")
        sents.append(line)
        i_start += 1
        if i_start == N:
            print('End of anns list')
            break
        start_ann = anns_sorted['start'][i_start]
        
    start_sent =  end_sent + 1

Target sent detected: 
                         Myopia.
 | 0 | 267 | Myopia

Target sent detected: 
 Review:            18 months with low vision aid assessment.
 | 1 | 344 | low vision

Target sent detected: 
                          Myopia.
 | 2 | 912 | Myopia

Target sent detected: 
 		Early lens opacities
 | 3 | 1543 | lens opacities

Target sent detected: 
                         Myopia.
 | 4 | 1582 | Myopia

End of anns list


In [11]:
# Result
sents

['                        Myopia.\n',
 'Review:            18 months with low vision aid assessment.\n',
 '                         Myopia.\n',
 '\t\tEarly lens opacities\n',
 '                        Myopia.\n']